In [1]:
import yfinance as yf 
import pandas as pd 
import numpy as np 
import datetime as dt

In [2]:

from google.oauth2 import service_account
import pandas as pd
import pandas_gbq as gbq
import json
from pandas_gbq import to_gbq

credentials_path = '../token.json'

# Authenticate with your credentials
credentials = service_account.Credentials.from_service_account_file(
    credentials_path, scopes=['https://www.googleapis.com/auth/bigquery'])

# Set the credentials for pandas_gbq
gbq.context.credentials = credentials

In [9]:

df = pd.read_csv("YFinance_data_cleaned.csv")

with open('../token.json', 'r') as token_file:
    token_data = json.load(token_file)
    project_id = token_data.get('project_id', 'default-project-id')

dataset_id = "raw_data"
table_id = 'yfinance_data'

to_gbq(df, destination_table=f'{project_id}.{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

C:\Users\shiho\AppData\Local\Temp\ipykernel_3000\3123296323.py:1: DtypeWarning: Columns (4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("YFinance_data_cleaned.csv")


ArrowTypeError: Expected bytes, got a 'float' object

In [8]:
df['High'] = df['High'].astype(float)
df['Low'] = df['Low'].astype(float)
df['Open'] = df['Open'].astype(float)
df['Close'] = df['Close'].astype(float)

ValueError: could not convert string to float: '33.7494991.8266666730244956'

# Loading in the values from YFinance data, which was aggregated monthly, then calculating the rolling mean of the volume

In [103]:

df = pd.read_csv("YFinance_data_cleaned.csv")
df['date'] = pd.to_datetime(df['yyyy'].astype(str) + df['mm'].astype(str), format='%Y%m')
df.sort_values(by=['tic', 'date'], inplace=True)
results = pd.DataFrame()
for ticker, group in df.groupby('tic'):
    rolling_window = group.rolling('365D', on='date')
    group['rolling_mean'] = rolling_window['Volume'].mean()

    results = pd.concat([results, group])

results['1YearVolumeRatio'] = results['Volume']/results['rolling_mean']

         Unnamed: 0  yyyy  mm       Open       High        Low      Close  \
1222632     1222632  2000   1  48.918097  49.765290  47.004649  48.319027   
1222633     1222633  2000   2  60.224427  63.608727  58.952521  61.807046   
1222634     1222634  2000   3  87.360826  91.856767  82.689944  87.663568   
1222635     1222635  2000   4  67.213030  70.281323  63.478842  66.965967   
1222636     1222636  2000   5  56.936046  59.407108  54.286156  56.341141   
...             ...   ...  ..        ...        ...        ...        ...   
1595585     1595585  2023   1   0.183640   0.183640   0.183640   0.183640   
1595586     1595586  2023   2   0.219100   0.219100   0.219100   0.219100   
1595587     1595587  2023   3   0.182887   0.182887   0.182887   0.182887   
1595588     1595588  2023   4   0.190000   0.190000   0.190000   0.190000   
1595589     1595589  2023   5   0.159500   0.159500   0.159500   0.159500   

         Adj Close        Volume    tic       date  rolling_mean  
1222632 

# Calculating RSI using Yfinance Data

In [179]:
import pandas as pd


df = pd.read_csv("YFinance_data_cleaned.csv")
df['date'] = pd.to_datetime(df['yyyy'].astype(str) + df['mm'].astype(str), format='%Y%m')
df.sort_values(by=['tic', 'date'], inplace=True)
def calculate_monthly_rsi(df):
    df['diff'] = df['Adj Close'].diff(1)
    df['gain'] = df['diff'].clip(lower=0).round(2).fillna(0)
    df['loss'] = df['diff'].clip(upper=0).abs().round(2).fillna(0)
    window_length = 12
    df['avg_gain'] = df['gain'].rolling(window=window_length, min_periods=1).mean()[:window_length+1]
    df['avg_loss'] = df['loss'].rolling(window=window_length, min_periods=1).mean()[:window_length+1]
    for i, row in enumerate(df['avg_gain'].iloc[window_length+1:]):
        df['avg_gain'].iloc[i + window_length + 1] =(df['avg_gain'].iloc[i + window_length] *(window_length - 1) +df['gain'].iloc[i + window_length + 1])/ window_length
    for i, row in enumerate(df['avg_loss'].iloc[window_length+1:]):
        df['avg_loss'].iloc[i + window_length + 1] =(df['avg_loss'].iloc[i + window_length] * (window_length - 1) + df['loss'].iloc[i + window_length + 1])/window_length
        df['rs'] = df['avg_gain'] / df['avg_loss']
        df['rsi'] = 100 - (100 / (1.0 + df['rs']))
    return df

rsi_data = df.groupby('tic').apply(calculate_monthly_rsi)

print(rsi_data)


               Unnamed: 0  yyyy  mm       Open       High        Low  \
tic                                                                    
A     1222632     1222632  2000   1  48.918097  49.765290  47.004649   
      1222633     1222633  2000   2  60.224427  63.608727  58.952521   
      1222634     1222634  2000   3  87.360826  91.856767  82.689944   
      1222635     1222635  2000   4  67.213030  70.281323  63.478842   
      1222636     1222636  2000   5  56.936046  59.407108  54.286156   
...                   ...   ...  ..        ...        ...        ...   
ZZZOF 1595585     1595585  2023   1   0.183640   0.183640   0.183640   
      1595586     1595586  2023   2   0.219100   0.219100   0.219100   
      1595587     1595587  2023   3   0.182887   0.182887   0.182887   
      1595588     1595588  2023   4   0.190000   0.190000   0.190000   
      1595589     1595589  2023   5   0.159500   0.159500   0.159500   

                   Close  Adj Close        Volume    tic       

In [183]:
rsi_data1 = rsi_data.drop(columns=['tic'])
rsi_data= rsi_data1.reset_index().dropna()
#Here only the first column is dropped cos you cant have RSI on the first data
rsi_data.to_pickle("rsi_data.pkl")

In [192]:
rsi_data = rsi_data.rename(columns={'yyyy': 'year', 'mm':'month'})
cri = pd.read_pickle("cri_compustat_merged_v5.pkl")
cri1 = cri.merge(rsi_data, on=['year','month','tic'], how='inner')
cri1.to_pickle("cri_yfinance_RSI.pkl")

In [202]:
def post_Processing(df, cri):
    df1 = df.drop(columns=['tic'])
    df = df1.reset_index().dropna()
    df = df.rename(columns={'yyyy':'year', 'mm':'month'})
    cri1 = cri.merge(rsi_data, on=['year','month','tic'], how='inner')
    return cri1

Stock Market Indicators:
Stock volatility.
Beta (compared to a global market index like MSCI World Index).
Trading volume.
Stock price momentum.


these features im getting since our data is monthly then ill be taking monthly averages right or do we want High & Low & mean also

In [17]:
def download_data(ticker, _start =dt.date(2000,1,1), _end = dt.date(2023,5,31)):
    return yf.download(ticker, start=_start, end = _end)
    

In [ ]:
company_PV_data = pd.DataFrame()
unique_Tickers = compustat_data_ALL['tic'].unique()
for i in unique_Tickers:
    try:
        temp_df = download_data(i) 
        temp_df.reset_index(inplace=True)
        temp_df['yyyy'] = temp_df.Date.apply(lambda x: x.year)
        temp_df['mm'] = temp_df.Date.apply(lambda x: x.month)
        temp_df.drop(['Date'],axis=1, inplace=True)
        grouped = temp_df.groupby(by = ['yyyy','mm']).mean()
        grouped.reset_index(inplace=True)
        grouped['tic'] = i
        company_PV_data = pd.concat([company_PV_data, grouped])
    except Exception as e:
        print(e)


In [193]:
cri.head()

,CompNo,year,month,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,...,financial_leverage_ratio,net_profit_margin,roa,asset_turnover,inventory_turnover,inventory_days,receivables_turnover,working_capital_turnover,pe_ratio,retention_ratio
0,37610,2000.0,1,0.089239,0.011089,0.087376,-0.524732,0.906364,-0.071829,0.000292,...,6.037135,24.982837,-12.570935,0.255210,-0.413,-0.413,1.912130,2.055384,-0.341530,6.787842
1,49536,2000.0,1,0.313050,0.007314,3.693176,0.625516,1.872712,1.010148,-0.069998,...,0.861522,75.426439,-8.322594,0.358015,0.000,0.000,0.574579,0.038721,-35.606061,-73.867110
2,32417,2000.0,1,0.089239,0.011089,6.165654,0.863910,1.275166,0.184606,0.013360,...,0.707156,52.341576,3.588386,0.236883,-4.463,-4.463,1.295937,0.525214,15.348538,183.313219
3,44933,2000.0,1,0.089239,0.011089,0.629410,-0.874122,0.000000,0.000000,0.004436,...,0.797336,74.952502,0.577149,0.290137,0.000,0.000,10.253247,-1.942129,12.797619,5.403687
4,39333,2000.0,1,0.089239,0.011089,4.337480,-0.245604,2.227256,-0.159957,-0.074377,...,0.998722,-157.397852,-27.807167,0.285404,0.000,0.000,1.291270,0.000000,-4.105839,-161.047245


In [194]:
df.head()

,Unnamed: 0,yyyy,mm,Open,High,Low,Close,Adj Close,Volume,tic,date
1222632,1222632,2000,1,48.918097,49.765290,47.004649,48.319027,41.553350,2.244042e+06,A,2000-01-01
1222633,1222633,2000,2,60.224427,63.608727,58.952521,61.807046,53.152766,1.646900e+06,A,2000-02-01
1222634,1222634,2000,3,87.360826,91.856767,82.689944,87.663568,75.388830,2.507812e+06,A,2000-03-01
1222635,1222635,2000,4,67.213030,70.281323,63.478842,66.965967,57.589325,2.424478e+06,A,2000-04-01
1222636,1222636,2000,5,56.936046,59.407108,54.286156,56.341141,48.452196,4.910608e+06,A,2000-05-01


In [204]:
def calculate_monthly_MACD(df):
    k = df['Close'].ewm(span=12, adjust=False, min_periods=0).mean()
    # Get the 12-day EMA of the closing price
    d = df['Close'].ewm(span=26, adjust=False, min_periods=0).mean()
    # Subtract the 26-day EMA from the 12-Day EMA to get the MACD
    macd = k - d
    # Get the 9-Day EMA of the MACD for the Trigger line
    macd_s = macd.ewm(span=9, adjust=False, min_periods=0).mean()
    # Calculate the difference between the MACD - Trigger for the Convergence/Divergence value
    macd_h = macd - macd_s
    # Add all of our new values for the MACD to the dataframe
    df['macd'] = df.index.map(macd)
    df['macd_h'] = df.index.map(macd_h)
    df['macd_s'] = df.index.map(macd_s)
    pd.set_option("display.max_columns", None)
    return df

In [278]:

df = pd.read_csv("YFinance_data_cleaned.csv")
MACD_data = df.groupby('tic').apply(calculate_monthly_MACD)
# MACD = post_Processing(MACD_data, cri)
# MACD.to_pickle("MACD.pkl")


In [218]:
bond_yield = pd.read_csv("United States 10-Year Bond Yield Historical Data.csv")

In [222]:
bond_yield.head()

,Date,Price,Open,High,Low,Change %,date
0,10/01/2023,4.901,4.599,5.023,4.531,7.03%,2023-10-01
1,09/01/2023,4.579,4.114,4.688,4.060,11.52%,2023-09-01
2,08/01/2023,4.106,3.965,4.366,3.949,3.51%,2023-08-01
3,07/01/2023,3.967,3.845,4.094,3.727,3.29%,2023-07-01
4,06/01/2023,3.841,3.639,3.892,3.570,5.33%,2023-06-01


In [219]:
bond_yield['date'] = pd.to_datetime(bond_yield['Date'])


Unnamed: 0  yyyy  mm       Open       High        Low      Close  \
tic                                                                             
A   1222632     1222632  2000   1  48.918097  49.765290  47.004649  48.319027   
    1222633     1222633  2000   2  60.224427  63.608727  58.952521  61.807046   
    1222634     1222634  2000   3  87.360826  91.856767  82.689944  87.663568   
    1222635     1222635  2000   4  67.213030  70.281323  63.478842  66.965967   
    1222636     1222636  2000   5  56.936046  59.407108  54.286156  56.341141   

             Adj Close        Volume tic       date      macd    macd_h  \
tic                                                                       
A   1222632  41.553350  2.244042e+06   A 2000-01-01  0.000000  0.000000   
    1222633  53.152766  1.646900e+06   A 2000-02-01  1.075967  0.860774   
    1222634  75.388830  2.507812e+06   A 2000-03-01  3.969328  3.003308   
    1222635  57.589325  2.424478e+06   A 2000-04-01  4.539882  2.859089   
    1222636  48.452196  4.910608e+06   A 2000-05-01  4.087595  1.925442   

               macd_s  
tic                    
A   1222632  0.000000  
    1222633  0.215193  
    1222634  0.966020  
    1222635  1.680793  
    1222636  2.162153

In [220]:
df = df.merge(bond_yield, on=['date'], how='inner')

In [221]:
df

,Unnamed: 0,yyyy,mm,Open_x,High_x,Low_x,Close,Adj Close,Volume,tic,date,Date,Price,Open_y,High_y,Low_y,Change %
0,1222633,2000,2,60.224427,63.608727,58.952521,61.807046,53.152766,1.646900e+06,A,2000-02-01,02/01/2000,6.409,6.409,6.409,6.409,-3.80%
1,651935,2000,2,85.261444,87.195109,83.707004,85.084973,63.259624,2.154457e+06,AA,2000-02-01,02/01/2000,6.409,6.409,6.409,6.409,-3.80%
2,1285016,2000,2,1.117850,1.144650,1.089050,1.117850,1.117850,1.090000e+04,AACS,2000-02-01,02/01/2000,6.409,6.409,6.409,6.409,-3.80%
3,1053301,2000,2,255.687500,275.062500,238.125000,260.937500,44.665585,3.829250e+04,AAIC,2000-02-01,02/01/2000,6.409,6.409,6.409,6.409,-3.80%
4,325786,2000,2,9.670000,9.918800,9.559400,9.775050,9.775050,5.885000e+03,AAIIQ,2000-02-01,02/01/2000,6.409,6.409,6.409,6.409,-3.80%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829787,1642924,2023,5,21.768095,22.031428,21.485238,21.789524,21.789524,1.216124e+06,ZWS,2023-05-01,05/01/2023,3.646,3.450,3.859,3.296,6.21%
1829788,758003,2023,5,9.103333,9.298809,8.813238,9.039524,9.039524,4.852619e+05,ZYME,2023-05-01,05/01/2023,3.646,3.450,3.859,3.296,6.21%
1829789,592370,2023,5,0.347133,0.358990,0.332038,0.344119,0.344119,2.226590e+05,ZYNE,2023-05-01,05/01/2023,3.646,3.450,3.859,3.296,6.21%
1829790,1228080,2023,5,10.182381,10.482381,9.804762,10.191905,10.191905,6.347000e+05,ZYXI,2023-05-01,05/01/2023,3.646,3.450,3.859,3.296,6.21%


# Wrong to change :( 
    - 1 year/ 2 year i/r can add as input features

In [324]:
def calculate_sortino_ratio(df):
    window = 12  # 12-month window

    # Calculate returns from adjusted close prices
    df['Returns'] = df['Adj Close'].pct_change()

    # Fill or replace None or NaN values in 'Returns' with 0
    df['Returns'].fillna(0, inplace=True)

    def sortino_ratio(returns):
        returns.apply(lambda x: 0 if x<0 else x )
        downside_deviation = np.std(returns)

        average_return = returns.mean()
        sortino = (average_return ) / downside_deviation if downside_deviation != 0 else 0
        return sortino

    df['Rolling_Sortino'] = df['Returns'].rolling(window).apply(sortino_ratio)

    return df

result = df.groupby('tic').apply(calculate_sortino_ratio)
print(result)

               Unnamed: 0  yyyy  mm       Open       High        Low  \
tic                                                                    
A     1222632     1222632  2000   1  48.918097  49.765290  47.004649   
      1222633     1222633  2000   2  60.224427  63.608727  58.952521   
      1222634     1222634  2000   3  87.360826  91.856767  82.689944   
      1222635     1222635  2000   4  67.213030  70.281323  63.478842   
      1222636     1222636  2000   5  56.936046  59.407108  54.286156   
...                   ...   ...  ..        ...        ...        ...   
ZZZOF 1595585     1595585  2023   1   0.183640   0.183640   0.183640   
      1595586     1595586  2023   2   0.219100   0.219100   0.219100   
      1595587     1595587  2023   3   0.182887   0.182887   0.182887   
      1595588     1595588  2023   4   0.190000   0.190000   0.190000   
      1595589     1595589  2023   5   0.159500   0.159500   0.159500   

                   Close  Adj Close        Volume    tic   Retu

In [234]:
sortino = post_Processing(result, cri)

In [252]:
sortino

,CompNo,year,month,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,ni2tatrend,sizelevel,sizetrend,m2b,sigma,liqfinlevel,lqfintrend,DTDmedianFin,DTDmedianNonFin,dummy297fin,Default,day,date_x,StartDate,EventDate,Duration,gvkey,tic,curr_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_equity_ratio,equity_ratio,financial_leverage_ratio,net_profit_margin,roa,asset_turnover,inventory_turnover,inventory_days,receivables_turnover,working_capital_turnover,pe_ratio,retention_ratio,level_1,Unnamed: 0,Open,High,Low,Close,Adj Close,Volume,date_y,diff,gain,loss,avg_gain,avg_loss,rs,rsi
0,29512,2000.0,2,0.102594,0.011443,6.227570,2.795305,3.179917,-0.141928,-0.008898,-0.004753,-0.846192,-0.135558,0.687133,0.122335,0.000000,0.000000,0.000000,2.670099,0,0,1,2000-02-01,1988-01-04,2000-02-29,4439 days,3806.0,DWSN,11.753546,11.753546,0.413121,18.195,0.000000,0.000000,2.276261,0.439317,-11.135612,-7.010221,1.000000,0.000,0.000,0.851301,0.348942,-4.166667,4.742000,68297,68297,2.363650,2.466762,2.214534,2.274815,2.227879,8.150550e+03,2000-02-01,0.298293,0.30,0.00,0.150000,0.000000,inf,100.000000
1,30452,2000.0,2,0.102594,0.011443,0.364351,0.264010,0.222074,0.130092,-0.005385,0.041543,-2.689560,0.093264,0.790459,0.390379,0.000000,0.000000,0.000000,2.670099,0,0,1,2000-02-01,1988-01-04,2000-02-29,4439 days,10380.0,SSY,1.462785,1.215043,-0.117046,4.713,0.103041,0.190897,0.539773,1.852630,5.209223,-2.167616,0.335546,-0.001,-0.001,1.508697,1.490770,-6.250000,5.434628,224166,224166,1.575000,1.637500,1.500000,1.603125,1.603125,5.120000e+03,2000-02-01,-0.134375,0.00,0.13,0.000000,0.065000,0.000000,0.000000
2,38757,2000.0,2,0.102594,0.011443,2.944128,0.163762,0.856195,-0.026859,0.005194,0.005101,1.469348,0.316466,3.547361,0.241964,0.000000,0.000000,0.000000,2.670099,0,0,1,2000-02-01,1995-10-20,2000-02-29,1593 days,61439.0,RSYS,4.572607,3.646266,1.885458,205.357,0.369775,0.541964,0.682287,1.465658,11.385208,2.988596,0.242145,-11.873,-11.873,1.208848,0.401705,15.972222,84.538815,934244,934244,45.767625,47.789100,44.681250,46.078125,46.078125,2.023550e+05,2000-02-01,2.409300,2.41,0.00,1.205000,0.000000,inf,100.000000
3,30034,2000.0,2,0.102594,0.011443,4.200410,-1.458280,1.237874,-0.222643,0.005541,0.001830,0.928959,-0.369168,0.908746,0.128692,0.000000,0.000000,0.000000,2.670099,0,0,1,2000-02-01,1988-01-04,2000-02-29,4439 days,5523.0,HVT,3.098744,1.956910,0.015641,200.472,0.575587,0.949792,0.606013,1.650130,50.751932,1.825898,0.257228,-14.048,-14.048,1.015314,0.889935,7.538168,529.155460,109796,109796,10.135938,10.265625,9.946875,10.112500,4.394574,9.010500e+04,2000-02-01,-0.741083,0.00,0.74,0.000000,0.370000,0.000000,0.000000
4,31496,2000.0,2,0.102594,0.011443,1.171534,1.517966,0.000000,0.000000,0.001990,-0.000172,0.045456,-0.030803,0.599335,0.106575,-5.461216,-0.254052,1.482047,0.000000,1,0,1,2000-02-01,1992-06-15,2000-02-29,2815 days,24825.0,OFC,1.470097,1.020431,0.000022,6.643,2.848940,1.220596,3.135287,0.318950,51.955896,0.514193,0.262028,0.000,0.000,2.395323,0.263435,16.562500,-40.969556,581123,581123,7.873075,7.947125,7.815775,7.897100,2.832198,1.935000e+04,2000-02-01,0.007801,0.01,0.00,0.005000,0.000000,inf,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598093,47250,2023.0,5,0.011539,0.009881,2.928379,-0.201082,0.226489,-0.101875,0.011788,-0.005447,3.374120,-0.304508,0.717161,0.116362,0.000000,0.000000,0.000000,2.934790,0,0,1,2023-05-01,2004-10-25,2023-05-31,6792 days,162129.0,MOS,1.132712,0.490247,-0.053623,685.800,0.438480,0.215793,2.031947,0.492139,25.341953,1.902220,1.000000,241.200,241.200,2.059600,5.255614,5.685254,11890.500000,1410447,1410447,37.417142,37.794762,36.427143,37.026190,37.026190,4.255876e+06,2023-05-01,-8.091705,0.00,8.09,1.750244,2.427474,0.721015,41.894747
598094,28391,2023.0,5,0.0115

In [269]:
rsi_data1.reset_index(inplace=True)

In [279]:


MACD_data = MACD_data[['yyyy', 'mm', 'tic', 'macd', 'macd_h', 'macd_s']]

In [325]:
sortino = result[['yyyy', 'mm','tic','Rolling_Sortino']]

In [270]:
rsi_data = rsi_data1[['yyyy', 'mm','tic','rsi']]

In [235]:
sortino.to_pickle('sortino.pkl')

In [327]:
sortino.drop(columns=['tic'], inplace=True)
sortino.reset_index(inplace=True)

/var/folders/4k/6rq2dc2j4w9fvhjkkjt2x0mr0000gn/T/ipykernel_65877/747720223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sortino.drop(columns=['tic'], inplace=True)


In [286]:
 MACD_data.drop(columns=['level_1'] ,inplace=True)
 MACD_data.reset_index(inplace=True)

In [ ]:
def post_Processing(df, cri):
    df1 = df.drop(columns=['tic'])
    df = df1.reset_index().dropna()
    df = df.rename(columns={'yyyy':'year', 'mm':'month'})
    cri1 = cri.merge(rsi_data, on=['year','month','tic'], how='inner')
    return cri1

In [328]:
final_df = sortino.merge(MACD_data, on = ['yyyy', 'mm', 'tic'], how = 'inner')

In [329]:
final_df = final_df.merge(rsi_data,  on = ['yyyy', 'mm', 'tic'], how = 'inner')

In [332]:
final_df = final_df.drop_na

AttributeError: 'DataFrame' object has no attribute 'drop_na'

In [330]:
test = pd.read_pickle('cri_yfinance.pkl')

In [331]:
final_df = final_df.merge(test,  on = ['year', 'month', 'tic'], how = 'inner')

KeyError: 'year'

In [333]:
final_df

,tic,level_1_x,yyyy,mm,Rolling_Sortino,level_1_y,macd,macd_h,macd_s,rsi
0,A,1222632,2000,1,NaN,1222632,0.000000,0.000000,0.000000,NaN
1,A,1222633,2000,2,NaN,1222633,1.075967,0.860774,0.215193,100.000000
2,A,1222634,2000,3,NaN,1222634,3.969328,3.003308,0.966020,100.000000
3,A,1222635,2000,4,NaN,1222635,4.539882,2.859089,1.680793,65.530596
4,A,1222636,2000,5,NaN,1222636,4.087595,1.925442,2.162153,55.676209
...,...,...,...,...,...,...,...,...,...,...
1832582,ZZZOF,1595585,2023,1,-0.027464,1595585,-0.612086,0.180779,-0.792866,31.538242
1832583,ZZZOF,1595586,2023,2,-0.158575,1595586,-0.577128,0.172590,-0.749718,33.238700
1832584,ZZZOF,1595587,2023,3,-0.205893,1595587,-0.546051,0.162933,-0.708985,32.361822
1832585,ZZZOF,1595588,2023,4,-0.458724,1595588,-0.514913,0.155257,-0.670170,32.844994


In [242]:
test.drop(columns=['Open', 'Low', 'High', 'Close', 'Unnamed: 0', 'Volume'], inplace=True)

In [244]:
test.drop(columns=['Adj Close'], inplace=True)

In [246]:
final_df.head()

,CompNo_x,year,month,StkIndx_x,STInt_x,dtdlevel_x,dtdtrend_x,liqnonfinlevel_x,liqnonfintrend_x,ni2talevel_x,ni2tatrend_x,sizelevel_x,sizetrend_x,m2b_x,sigma_x,liqfinlevel_x,lqfintrend_x,DTDmedianFin_x,DTDmedianNonFin_x,dummy297fin_x,Default_x,day_x,date_x_x,StartDate_x,EventDate_x,Duration_x,gvkey_x,tic,curr_ratio_x,quick_ratio_x,cash_ratio_x,net_working_capital_x,debt_ratio_x,debt_equity_ratio_x,equity_ratio_x,financial_leverage_ratio_x,net_profit_margin_x,roa_x,asset_turnover_x,inventory_turnover_x,inventory_days_x,receivables_turnover_x,working_capital_turnover_x,pe_ratio_x,retention_ratio_x,level_1_x,Unnamed: 0_x,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,date_y_x,diff_x,gain_x,loss_x,avg_gain_x,avg_loss_x,rs_x,rsi_x,CompNo_y,StkIndx_y,STInt_y,dtdlevel_y,dtdtrend_y,liqnonfinlevel_y,liqnonfintrend_y,ni2talevel_y,ni2tatrend_y,sizelevel_y,sizetrend_y,m2b_y,sigma_y,liqfinlevel_y,lqfintrend_y,DTDmedianFin_y,DTDmedianNonFin_y,dummy297fin_y,Default_y,day_y,date_x_y,StartDate_y,EventDate_y,Duration_y,gvkey_y,curr_ratio_y,quick_ratio_y,cash_ratio_y,net_working_capital_y,debt_ratio_y,debt_equity_ratio_y,equity_ratio_y,financial_leverage_ratio_y,net_profit_margin_y,roa_y,asset_turnover_y,inventory_turnover_y,inventory_days_y,receivables_turnover_y,working_capital_turnover_y,pe_ratio_y,retention_ratio_y,level_1_y,Unnamed: 0_y,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y,date_y_y,diff_y,gain_y,loss_y,avg_gain_y,avg_loss_y,rs_y,rsi_y,level_1,Unnamed: 0,Open,High,Low,Close,Adj Close,Volume,date,diff,gain,loss,avg_gain,avg_loss,rs,rsi,CompNo,StkIndx,STInt,dtdlevel,dtdtrend,liqnonfinlevel,liqnonfintrend,ni2talevel,ni2tatrend,sizelevel,sizetrend,m2b,sigma,liqfinlevel,lqfintrend,DTDmedianFin,DTDmedianNonFin,dummy297fin,Default,day,date_x,StartDate,EventDate,Duration,gvkey,curr_ratio,quick_ratio,cash_ratio,net_working_capital,debt_ratio,debt_equity_ratio,equity_ratio,financial_leverage_ratio,net_profit_margin,roa,asset_turnover,inventory_turnover,inventory_days,receivables_turnover,working_capital_turnover,pe_ratio,retention_ratio,date_y,rolling_mean,1YearVolumeRatio
0,29512,2000.0,2,0.102594,0.011443,6.227570,2.795305,3.179917,-0.141928,-0.008898,-0.004753,-0.846192,-0.135558,0.687133,0.122335,0.000000,0.000000,0.000000,2.670099,0,0,1,2000-02-01,1988-01-04,2000-02-29,4439 days,3806.0,DWSN,11.753546,11.753546,0.413121,18.195,0.000000,0.000000,2.276261,0.439317,-11.135612,-7.010221,1.000000,0.000,0.000,0.851301,0.348942,-4.166667,4.742000,68297,68297,2.363650,2.466762,2.214534,2.274815,2.227879,8150.55,2000-02-01,0.298293,0.30,0.00,0.150,0.000,inf,100.0,29512,0.102594,0.011443,6.227570,2.795305,3.179917,-0.141928,-0.008898,-0.004753,-0.846192,-0.135558,0.687133,0.122335,0.000000,0.000000,0.000000,2.670099,0,0,1,2000-02-01,1988-01-04,2000-02-29,4439 days,3806.0,11.753546,11.753546,0.413121,18.195,0.000000,0.000000,2.276261,0.439317,-11.135612,-7.010221,1.000000,0.000,0.000,0.851301,0.348942,-4.166667,4.742000,68297,68297,2.363650,2.466762,2.214534,2.274815,2.227879,8150.55,2000-02-01,0.298293,0.30,0.00,0.150,0.000,inf,100.0,68297,68297,2.363650,2.466762,2.214534,2.274815,2.227879,8150.55,2000-02-01,0.298293,0.30,0.00,0.150,0.000,inf,100.0,29512,0.102594,0.011443,6.227570,2.795305,3.179917,-0.141928,-0.008898,-0.004753,-0.846192,-0.135558,0.687133,0.122335,0.000000,0.000000,0.000000,2.670099,0,0,1,2000-02-01,1988-01-04,2000-02-29,4439 days,3806.0,11.753546,11.753546,0.413121,18.195,0.000000,0.000000,2.276261,0.439317,-11.135612,-7.010221,1.000000,0.000,0.000,0.851301,0.348942,-4.166667,4.742000,2000-02-01,5094.65,1.599825
1,30452,2000.0,2,0.102594,0.011443,0.364351,0.264010,0.222074,0.130092,-0.005385,0.041543,-2.689560,0.093264,0.790459,0.390379,0.000000,0.000000,0.000000,2.670099,0,0,1,2000-02-01,1988-01-04,2000-02-29,4439 days,10380.0,SSY,1.462785,1.215043,-0.117046,4.713,0.103041,0.190897,0.539773,1.852630,5.209223,-2.167616,0.335546,-0.001,-0.001,1.508697,1.490770,-6.250000,5.434628,224166,224166,1.575000,1.637500,1.500000,1.60312

In [335]:
import pandas as pd

# Assuming 'df' is your DataFrame
columns = final_df.columns

filtered_columns = [col for col in columns if not (col.endswith('_x') or col.endswith('_y'))]

filtered_columns_list = list(filtered_columns)
print(filtered_columns_list)


['tic', 'yyyy', 'mm', 'Rolling_Sortino', 'macd', 'macd_h', 'macd_s', 'rsi']


In [337]:
final_df = final_df[filtered_columns_list]

In [296]:
final_df = final_df[['tic', 'yyyy', 'mm', 'Rolling_Sortino', 'macd', 'macd_h', 'macd_s', 'rsi']]

In [298]:
final_df.to_pickle('final_yfiance.pkl')

In [251]:
sortino.columns

Index(['CompNo', 'year', 'month', 'StkIndx', 'STInt', 'dtdlevel', 'dtdtrend',
       'liqnonfinlevel', 'liqnonfintrend', 'ni2talevel', 'ni2tatrend',
       'sizelevel', 'sizetrend', 'm2b', 'sigma', 'liqfinlevel', 'lqfintrend',
       'DTDmedianFin', 'DTDmedianNonFin', 'dummy297fin', 'Default', 'day',
       'date_x', 'StartDate', 'EventDate', 'Duration', 'gvkey', 'tic',
       'curr_ratio', 'quick_ratio', 'cash_ratio', 'net_working_capital',
       'debt_ratio', 'debt_equity_ratio', 'equity_ratio',
       'financial_leverage_ratio', 'net_profit_margin', 'roa',
       'asset_turnover', 'inventory_turnover', 'inventory_days',
       'receivables_turnover', 'working_capital_turnover', 'pe_ratio',
       'retention_ratio', 'level_1', 'Unnamed: 0', 'Open', 'High', 'Low',
       'Close', 'Adj Close', 'Volume', 'date_y', 'diff', 'gain', 'loss',
       'avg_gain', 'avg_loss', 'rs', 'rsi'],
      dtype='object')

In [339]:
final_df.to_pickle("yfinance_sortino_MACD_RSI.pkl")

In [300]:
final_df.rename(columns={'yyyy': 'year', 'mm':'month'}, inplace=True)

/var/folders/4k/6rq2dc2j4w9fvhjkkjt2x0mr0000gn/T/ipykernel_65877/2843897740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={'yyyy': 'year', 'mm':'month'}, inplace=True)


In [303]:
final_df = final_df.merge(test,  on = ['year', 'month', 'tic'], how = 'inner')

In [305]:
final_df.to_pickle('yfiannce_cri.pkl')
